<a href="https://colab.research.google.com/github/YusunPark/move_bike/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4_shortest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import requests
import json
import pandas as pd
import numpy as np
import datetime

In [2]:
# 자전거 데이터 호출
def call_bike():
  BIKE_API_KEY = '74796e6d72736f6c39314a50736843'

  bike_url_1 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/1/1000/'
  bike_url_2 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/1001/2000/'
  bike_url_3 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/2001/3000/'

  resp_bike_1 = requests.get(bike_url_1)
  bike_json_1 = json.loads(resp_bike_1.text)

  resp_bike_2 = requests.get(bike_url_2)
  bike_json_2 = json.loads(resp_bike_2.text)

  resp_bike_3 = requests.get(bike_url_3)
  bike_json_3 = json.loads(resp_bike_3.text)

  df1 = pd.DataFrame.from_dict(bike_json_1['rentBikeStatus']['row'])
  df2 = pd.DataFrame.from_dict(bike_json_2['rentBikeStatus']['row'])
  df3 = pd.DataFrame.from_dict(bike_json_3['rentBikeStatus']['row'])

  return pd.concat([df1, df2, df3])


In [3]:
data = call_bike()
data = data[['stationLatitude','stationLongitude','stationId']]
data.head()

In [ ]:
API_KEY = '5d1b586339b7fbdfcd3c1088fc54a215'

# 대형마트, 편의점, 학교, 학원, 지하철역, 문화시설
category_group_code = ['MT1', 'CS2', 'SC4', 'AC5', 'SW8', 'CT1']


for code in category_group_code[1:]:
  print(f'-------{code}-------')
  # 새 distance 생성
  distance = []
  for i in range(2595):

    bike_stop_y = data.iloc[i][0]
    bike_stop_x = data.iloc[i][1]

    url = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={code}&y={bike_stop_y}&x={bike_stop_x}&sort=distance"
    url_result = requests.get(url, params = {'category_group_code': code, 'page': 1, 'size' : 1}, headers={'Authorization' : 'KakaoAK ' + API_KEY}).json()
    distance.append(int(url_result['documents'][0]['distance']))

  data[code] = distance

-------CS2-------
-------SC4-------


In [ ]:
data.to_csv("distance_data.csv")